In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as dt
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from matplotlib import pyplot as plt
import plotly.express as px

sys.path.append("../src/")
from rf_et0 import *

# Loading data

In [2]:
# loading observed data
df_weather = pd.read_csv(os.path.join('../data/',"MITUNGU.csv"), sep=";", dtype={'CodeStation': object})
df_weather["Jour"] = df_weather.apply(lambda x: dt.strptime(x["Jour"], "%d/%m/%Y").date(), axis=1)
df_weather

,CodeStation,Jour,TMAX,TMIN,WIND,IRRAD,LAT,LON,ELEV,RAIN,VAP
0,MITUNG,2000-11-30,27.2,15.9,0.8,21200000.0,-0.08,37.75,1189,0.0,18.762895
1,MITUNG,2000-12-01,26.5,15.5,0.6,18900000.0,-0.08,37.75,1189,0.0,19.404029
2,MITUNG,2000-12-02,26.8,17.9,0.6,19400000.0,-0.08,37.75,1189,0.0,17.478381
3,MITUNG,2000-12-03,26.4,17.3,0.6,19200000.0,-0.08,37.75,1189,0.0,18.715514
4,MITUNG,2000-12-04,27.7,14.6,0.2,22100000.0,-0.08,37.75,1189,0.0,18.208180
...,...,...,...,...,...,...,...,...,...,...,...
361,MITUNG,2001-11-26,25.9,17.1,0.7,20100000.0,-0.08,37.75,1189,0.0,17.058108
362,MITUNG,2001-11-27,26.5,17.1,1.1,19400000.0,-0.08,37.75,1189,0.0,18.314274
363,MITUNG,2001-11-28,26.3,16.9,1.1,19600000.0,-0.08,37.75,1189,0.0,19.408111
364,MITUNG,2001-11-29,26.5,16.0,0.8,20100000.0,-0.08,37.75,1189,0.0,19.739134


# Training model

In [3]:
# site coordinates, elevation and maximum value for training dates interval are taken from the test dataset
lat = df_weather["LAT"].unique()[0]
lon = df_weather["LON"].unique()[0]
elev = df_weather["ELEV"].unique()[0]
max_train_date = df_weather["Jour"][-1:].values[0]

# training variables can be any combination of TMAX, TMIN, WIND, IRRAD, RAIN, VAP
training_variables = ["TMAX", "TMIN", "WIND", "IRRAD", "RAIN", "VAP"]

# performing data retrieval, model training and cross-validation
df_NASAPower, cv_results, xg_reg = train_RF_ET0_model(lat, lon, max_train_date, training_variables, elev)

> Retrieving NASA Power data
Weather data provided by: NASAPowerWeatherDataProvider
--------Description---------
NASA/POWER CERES/MERRA2 Native Resolution Daily Data
----Site characteristics----
Elevation: 1910.8
Latitude:  -0.080
Longitude: 37.750
Data available for 1984-01-01 - 2022-08-26
Number of missing days: 3

Model training will be performed using data ranging from 1984-01-01 to 2001-11-30 

> Exact elevation provided, calculating corrected ET0...
> Calculating cross-validation metrics...
> Training RF model...
> Done !


In [4]:
mean_rmse = cv_results['test-rmse-mean'][-1:].values[0]
std_rmse = cv_results['test-rmse-std'][-1:].values[0]
print("Mean RMSE for the predictions performed on each cross-validation batch is",np.round(mean_rmse,3),",")
print("with a square deviation value of",np.round(std_rmse,3),"(RSD =",np.round(std_rmse/mean_rmse*100,2),"%).")

Mean RMSE for the predictions performed on each cross-validation batch is 0.121 ,
with a square deviation value of 0.007 (RSD = 5.64 %).


# Performing ET0 inference, and comparing with Penman-Monteith ET0

In [8]:
# computing Penman-Monteith reference evapotranspiration (ET0)
angstA = 0.29
angstB = 0.49
df_weather["ET0-PM"] = df_weather.apply(lambda x: reference_ET(x["Jour"], x["LAT"], x["ELEV"], x["TMIN"], x["TMAX"], x["IRRAD"], x["VAP"], x["WIND"], angstA, angstB)[2], axis=1)

# performing inference
preds = xg_reg.predict(df_weather[training_variables])
df_weather["ET0-RF"] = preds

In [9]:
# scatterplot of ET0-PM vs inferred ET0-RF 
fig = px.scatter(df_weather, x="ET0-PM", y="ET0-RF", trendline="ols")
fig.show()

In [17]:
# time-series of ET0-PM vs inferred ET0-RF 
fig = px.line(df_weather, x='Jour', y=["ET0-PM","ET0-RF"])
fig.show()

In [11]:
print("coefficient of determination",r2_score(df_weather["ET0-PM"], df_weather["ET0-RF"]))
print("MAE",mean_absolute_error(df_weather["ET0-PM"], df_weather["ET0-RF"]))
print("RMSE",mean_squared_error(df_weather["ET0-PM"], df_weather["ET0-RF"], squared=False))


coefficient of determination 0.5965835955049716
MAE 0.3470330743440361
RMSE 0.40631998000873965
